In [1]:
import pandas as pd
import wm_helpers

In [2]:
matches = pd.read_csv("data/raw/HistoricMatches.csv")

matches = matches[["Year", "Home Team Name", "Home Team Goals", "Away Team Name", "Away Team Goals", "Stage", "Win conditions"]]
matches = matches.query('Year >= 2010')

replace_dict = {'rn">Serbia and Montenegro':'Serbia and Montenegro',
                "C�te d'Ivoire":"Côte d'Ivoire",
                'rn">Bosnia and Herzegovina':'Bosnia and Herzegovina',
                'rn">Trinidad and Tobago':'Trinidad and Tobago'}

matches["Home Team Name"] = matches["Home Team Name"].replace(replace_dict)
matches["Away Team Name"] = matches["Away Team Name"].replace(replace_dict)

matches["Group Stage"] = matches["Stage"].str.contains("Group")
matches = matches.drop(["Stage"], axis=1)

teams = pd.read_csv("data/raw/TeamsInfo.csv")
teams["Nationality"] = teams["Nationality"].replace({'United States':'USA', "Iran":"IR Iran"})

In [3]:
teams = pd.read_csv("data/raw/TeamsInfo.csv")
teams["Nationality"] = teams["Nationality"].replace({'United States':'USA', "Iran":"IR Iran"})

fifa = pd.read_csv("data/clean/fifa_ranking_wm.csv")

for year in [2010, 2014]:
    matches_y = matches[matches["Year"] == year]
    fifa_y = fifa[fifa["year"] == year]
    teams_y = teams[teams["Year"] == year]

    print(len(teams))
    derived_teams = wm_helpers.derive_scores(matches_y["Home Team Name"], fifa_y, teams_y)
    teams = teams.append(derived_teams)
    print(len(teams))
teams.to_csv("data/clean/teams_wm.csv", index=False)

138
148
148
157


In [4]:
import re

fifa = pd.read_csv("data/clean/fifa_ranking_wm.csv")

df_all = pd.merge(matches, teams, how="left", left_on=["Year", "Home Team Name"], 
              right_on=["Year", "Nationality"],)
df_all = pd.merge(df_all, teams, how="left", left_on=["Year", "Away Team Name"], 
              right_on=["Year", "Nationality"], suffixes=(' Home', ' Away'))

df_all = pd.merge(df_all, fifa[["year", "country_full", "rank"]], how="left", left_on=["Year", "Home Team Name"], 
              right_on=["year", "country_full"])
df_all = pd.merge(df_all, fifa[["year", "country_full", "rank"]], how="left", left_on=["Year", "Away Team Name"], 
              right_on=["year", "country_full"], suffixes=(' Home', ' Away'))

df_all.drop(['year Away', 'year Home', "country_full Away", "country_full Home"], axis=1, inplace=True)

def add_penalties_goals(row):
    win_text = row["Win conditions"]
    if (win_text != '') & ("penalties" in win_text):
        m = re.search("(?:\()(\d+)(?:\s-\s)(\d+)(?:\))", win_text)
        row["Home Team Goals"] = m.group(1)
        row["Away Team Goals"] = m.group(2)
    return row  
        
df_final = df_all.apply(add_penalties_goals, axis=1)
df_final.drop(["Win conditions"], axis=1, inplace=True)

df_final.to_csv("data/clean/training_data.csv", index=False)

In [6]:
#teams = pd.read_csv("data/raw/TeamsInfo.csv")
teams2018 = teams[teams["Year"] == 2018]

new_matches =  pd.read_csv("data/raw/2018Matches.csv")
new_matches = new_matches[new_matches["Round Number"].isin(["1","2","3"])]
new_matches["Home Team Name"] = new_matches["Home Team Name"].replace({"Iran":"IR Iran"})
new_matches["Away Team Name"] = new_matches["Away Team Name"].replace({"Iran":"IR Iran"})

new_matches.to_csv("data/clean/2018MatchesGroup.csv", index=False)

#fifa = pd.read_csv("data/raw/fifa_ranking_wm.csv")
fifa_2018 = fifa[fifa["year"] == 2018]
print(len(teams2018))
derived_teams = wm_helpers.derive_scores(new_matches["Home Team Name"], fifa_2018, teams2018)
teams2018 = teams2018.append(derived_teams)
print(len(teams2018))
teams2018.to_csv("data/clean/teams2018.csv", index=False)

50
61
